# Hyper parameter tunning
- updated 0922
- data  
7th_FE_ver4  
np.log1p()
np.expm1()

catboost 튜닝
- optuna, bayesian

## y_predict 값들 피클로 저장해놓기   
## ISSUE : get dummies 이용
stacking & scaler 구현


## ready

In [10]:
cd /content/drive/My Drive/빅콘

/content/drive/My Drive/빅콘


In [11]:
!apt-get update --qq
!apt-get install fonts-nanum* -qq

E: Command line option --qq is not understood in combination with the other options


In [12]:
# 한글 폰트
import matplotlib.font_manager as fm
fm._rebuild()

In [13]:
# # CUDA
# !pip install mxnet-cu80
# import mxnet as mx

In [14]:
# ready
import os
import sys
import joblib
import pickle
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd 
import numpy as np
import random
import math

from IPython.display import display
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_info_columns', 500)

import imp
import datetime
from tqdm import tqdm

In [15]:
# vusualize
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)

In [16]:
# sklearn
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score

In [17]:
!pip install shap
import shap

### Load data

In [18]:
# load data
data = joblib.load(os.path.join('data', '7th_train_FE.pkl'))
locals().update(data)

# X, y 설정
X = data['X'] ; y = data['y'] ; log_y = np.log1p(y)

In [19]:
# na
X.isnull().sum().sum()

3733

In [20]:
# fill na
X0 = X.fillna(0)

# One Hot
X_dum = pd.get_dummies(X)
print(X.shape, X_dum.shape)
# 모두 더미화 되어있음

(35379, 402) (35379, 402)


### MAPE + CV function

In [21]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [22]:
# mape
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [23]:
# mape with exp
def MAPE_exp(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((np.expm1(y_true) - np.expm1(y_pred)) / np.expm1(y_true))) * 100

# Stacking + SHAP
#### CV + Save Prediction + SHAP

In [24]:
# stacking with cv=5
# MAPE_exp_cv 구현

def MAPE_exp_st(model, X, y, cv_splits, version, scaling=False):        
    # setting
    mape = {'val_mape' : [], 'test_mape' : [],'final_mape' : []}
    pred = {'val_idx' : [], 'val_pred' : [],
            'test_idx' : [], 'test_pred' : [],
            'final_pred' : []}      # final : test set mean 값
    # SHAP
    SHAP = {'shap_values' : [], 'expected_value' : []}

    # split train, test
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=77)
    pred['test_idx'].append(X_test.index)
    
    # for 문 돌리기
    cv=KFold(n_splits=cv_splits,random_state=77,shuffle=True)
    for t,v in cv.split(X_train):
        X_train_ = X_train.iloc[t]       # 훈련용
        X_val_ = X_train.iloc[v]         # 검증용 분리.
        y_train_ = y_train.iloc[t]
        y_val_ = y_train.iloc[v]

        pred['val_idx'].append(v)

        # scaling : MinMax or Standard
        if scaling : 
            if scaling == 'MinMax':
                scaler = MinMaxScaler()
            elif scaling == 'Standard':
                scaler = StandardScaler()
            # fitting
            X_train_ = scaler.fit_transform(X_train_)
            X_val_ = scaler.transform(X_val_)
            X_test_ = scaler.transform(X_test)
        
        else:
            X_test_ = X_test

        # modeling
        model.fit(X_train_,y_train_)

        # predict
        y_pred = model.predict(X_val_)
        pred['val_pred'].append(np.expm1(y_pred))
        test_pred = model.predict(X_test_)
        pred['test_pred'].append(np.expm1(test_pred))

        # mape
        mape['val_mape'].append(MAPE_exp(y_val_, y_pred))
        mape['test_mape'].append(MAPE_exp(y_test,test_pred))

        #SHAP
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_test_)

        SHAP['shap_values'].append(shap_values)
        SHAP['expected_value'].append(explainer.expected_value)

    # final values
    final_test = np.mean(pred['test_pred'], axis=0)
    final_mape = np.mean(mape['test_mape'], axis=0)
    pred['final_pred'].append(final_test)
    mape['final_mape'].append(final_mape)
    
    # loss
    #loss = - np.mean(mape)

    # save pickle
    with open('mape_' + version + '.pickle', 'wb') as f:
        pickle.dump(mape, f, pickle.HIGHEST_PROTOCOL)
    with open('pred_' + version + '.pickle', 'wb') as f:
        pickle.dump(pred, f, pickle.HIGHEST_PROTOCOL)

    return mape, pred, SHAP

## Tunning Randomforest

### ready catboost

In [26]:
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=d153c62d9f4f4baa227241533092e90b980760f0bf2edea1efd5c7311c021c32
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [25]:
from sklearn.ensemble import RandomForestRegressor

### random forest + bayesian

In [ ]:
# Setting
%%time

X, y = X0, log_y 
def rf_eval(n_estimators ,    # 3h 30min
            max_depth,
            min_samples_split):
    
    n_splits=5
    cv = KFold(n_splits=5, shuffle=True, random_state=77)
    mape = []

    for tr_ind, val_ind in cv.split(X , y):
        X_train = X.iloc[tr_ind]
        y_train = y.iloc[tr_ind]
        X_valid = X.iloc[val_ind]
        y_valid = y.iloc[val_ind]

        # scaling
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_valid = scaler.transform(X_valid)
        
        model = RandomForestRegressor()
        model.fit(X_train, 
                  y_train)
        
        y_pred = model.predict(X_valid)
        mape.append(MAPE_exp(y_pred,y_valid))
    return -np.mean(mape)

rfBO = BayesianOptimization(rf_eval,
                             {"n_estimators": (10, 100),
                              "max_depth": (10, 150),
                              "min_samples_split": (2, 10)
                             },
                             random_state=77)     # 77
rfBO.maximize(init_points=5, n_iter=30)

|   iter    |  target   | max_depth | min_sa... | n_esti... |
-------------------------------------------------------------
|  1        | -37.31    |  138.7    |  7.138    |  77.83    |
|  2        | -37.29    |  29.5     |  2.699    |  80.92    |
|  3        | -37.4     |  55.66    |  6.329    |  31.62    |
|  4        | -37.29    |  86.36    |  5.204    |  74.37    |
|  5        | -37.28    |  127.1    |  6.708    |  36.65    |
|  6        | -37.4     |  150.0    |  2.0      |  10.0     |
|  7        | -37.2     |  127.9    |  9.455    |  35.12    |
|  8        | -37.37    |  10.0     |  10.0     |  10.0     |
|  9        | -37.39    |  60.63    |  10.0     |  100.0    |
|  10       | -37.36    |  10.0     |  10.0     |  100.0    |
|  11       | -37.23    |  105.1    |  10.0     |  10.0     |
|  12       | -37.35    |  10.0     |  10.0     |  55.14    |
|  13       | -37.31    |  121.4    |  9.608    |  99.91    |
|  14       | -37.42    |  149.9    |  9.647    |  98.5     |


|   iter    |  target   | max_depth | min_sa... | n_esti... |  
7        | -37.2     |  127.9    |  9.455    |  35.12 

In [ ]:
# best parameter
rfBO.max

### best Random Forest + SHAP
https://github.com/cryingjin/Shopping_GG/blob/jbeen2/Modeling/0915_Tuning2.ipynb

In [ ]:
'''
best_params = {
    'bagging_temperature': round(641.7274273159518),
    'border_count': round(39.73514206561576),
    'depth': round(8.785912134398988),
    'learning_rate': 0.0202,     #0.02017695398268462
    'min_data_in_leaf': round(2.7106426642135535)}
'''
# best catboost
rf_BO = RandomForestRegressor()

In [ ]:
# 평가 및 저장
mape_rfBO, pred_rfBO, SHAP_rfBO = MAPE_exp_st(rf_BO, X, log_y, 5, 'rfBO_0923', scaling='MinMax') 
# 2h + a

0:	learn: 0.0488429	total: 107ms	remaining: 17m 45s
5000:	learn: 0.0172931	total: 7m 36s	remaining: 7m 36s
9999:	learn: 0.0170381	total: 15m 29s	remaining: 0us
0:	learn: 0.0486737	total: 108ms	remaining: 18m 1s
5000:	learn: 0.0175891	total: 7m 43s	remaining: 7m 43s
9999:	learn: 0.0172854	total: 15m 35s	remaining: 0us
0:	learn: 0.0488426	total: 104ms	remaining: 17m 18s
5000:	learn: 0.0178003	total: 7m 45s	remaining: 7m 45s
9999:	learn: 0.0174229	total: 15m 40s	remaining: 0us
0:	learn: 0.0486540	total: 107ms	remaining: 17m 48s
5000:	learn: 0.0178197	total: 7m 42s	remaining: 7m 41s
9999:	learn: 0.0174812	total: 15m 31s	remaining: 0us
0:	learn: 0.0488669	total: 103ms	remaining: 17m 13s
5000:	learn: 0.0174321	total: 7m 35s	remaining: 7m 35s
9999:	learn: 0.0171273	total: 15m 17s	remaining: 0us


In [ ]:
mape_catBO

{'final_mape': [43.97830423209804],
 'test_mape': [43.15712847934601,
  43.48934052081531,
  44.47230947992317,
  43.91217366291737,
  44.860569017488366],
 'val_mape': [44.29497182444521,
  42.480774813138716,
  45.825055340301766,
  44.20043306253304,
  47.1510410278104]}

In [ ]:
pred_catBO

{'final_pred': [array([ 7337454.06888649, 12836978.76630708, 22969872.35931302, ...,
         36402360.29188809, 12124641.20711445, 11494854.33233659])],
 'test_idx': [Int64Index([ 8734,  7885,  2413, 12475,  4571,  9835, 35907, 14833, 10316,
              31140,
              ...
              25556, 30587,  6430, 11575, 33365, 14158, 28909, 34363, 26499,
               4615],
             dtype='int64', length=7076)],
 'test_pred': [array([ 7461435.2239652 , 10557717.41728055, 23894949.15386017, ...,
         35513894.20933883, 12120182.22411971, 11519053.78391614]),
  array([ 7601448.90771597, 13921693.12071874, 21934620.88420022, ...,
         39891709.87014625, 13856132.4276284 , 11470092.0453258 ]),
  array([ 7432880.10625217, 11227923.81908253, 22974984.89611312, ...,
         41359930.50105048, 10656494.10370523, 10639620.3791659 ]),
  array([ 7357800.3096538 , 12802896.78378526, 22298655.92877083, ...,
         33276259.39804833, 13573140.7957979 , 11543905.15723657]),
  array

In [ ]:
SHAP_catBO

{'expected_value': [16.576933918962283,
  16.580584109528008,
  16.57420666919765,
  16.577127688019747,
  16.574577619578996],
 'shap_values': [array([[ 4.63245740e-03,  5.87147233e-03,  1.17219703e-02, ...,
           7.41385555e-05, -7.25387629e-05,  2.40674398e-04],
         [ 2.98230024e-03, -9.60202776e-04,  5.75277243e-03, ...,
          -8.03367202e-06,  2.15578149e-04,  5.62376553e-05],
         [ 3.94514954e-03,  1.37590817e-02,  7.04159760e-03, ...,
           1.56692615e-04, -7.74543642e-05, -4.08986016e-04],
         ...,
         [ 9.65387222e-03,  2.24792065e-03,  7.36174504e-03, ...,
           1.53060624e-04,  1.54371323e-04, -7.75945689e-04],
         [ 6.81013197e-03,  7.25782624e-03,  1.34539664e-02, ...,
           2.81199496e-05,  3.98526642e-04, -1.06682736e-03],
         [ 4.24242564e-03,  1.43624443e-03, -6.92165065e-03, ...,
          -1.35605003e-05,  1.55674166e-04, -1.84271647e-03]]),
  array([[-9.76021623e-04, -3.46016304e-03, -1.51178849e-03, ...,
       

### Random Forest + OPTUNA

In [ ]:
! pip install optuna
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
def objective(trial, X, y):
    
    # 파라미터들의 search 범위
    # bayesian 과 동잂하게 설정한다
    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 150), # (5, 8.99)
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),  
        'n_estimators': trial.suggest_uniform('learning_rate', 0.001, 0.1)
    }

    # random forest setting
    rf_model = RandomForestRegressor(**params)
                                 #task_type = 'GPU')
    
    # cat boost : split + cv
    mape, pred, SHAP = MAPE_exp_st(rf_model, X0, log_y, 5, 'rfOP_0923', 'MinMax')
    return mape['final_mape'][0]

In [ ]:
%%time
rf_study = optuna.create_study()
rf_study.optimize(lambda x: objective(x, X0, log_y), timeout=1000, n_jobs=-1)
print(rf_study.best_params, rf_study.best_value)

# Random Forest tunning
### 1. rfBO
randomforest + bayesian -> 
### 2. catOP
randomforest + OPTUNA -> 
